# 💰 Fase 02: Financial & Business Logic Audit
Este notebook valida la consistencia de los datos contables y de inversión utilizando las reglas definidas en `config.yaml`.

In [1]:
# Celda 1: Configuración de Ambiente
import sys
import os
import pandas as pd
import json

# Agregar 'src' al path
sys.path.insert(0, os.path.abspath('..'))

from src.utils.encoding_patch import apply_utf8_patch
from src.loader import DataLoader
from src.validator import BusinessValidator

# Asegurar codificación UTF-8
apply_utf8_patch()

print("✅ Ambiente de Auditoría Financiera configurado.")

✅ Ambiente de Auditoría Financiera configurado.


In [2]:
# Celda 2: Carga de Datos Locales
# Cargamos los archivos Parquet que ya fueron descargados por el DataLoader
loader = DataLoader()
tables_to_audit = ["ventas_diarias", "redes_sociales"]
data = {}

for table in tables_to_audit:
    file_path = os.path.join("..", loader.config["general"]["paths"]["raw"], f"{table}.parquet")
    if os.path.exists(file_path):
        data[table] = pd.read_parquet(file_path)
        print(f"✅ Cargada tabla: {table} ({len(data[table])} registros)")
    else:
        print(f"❌ No se encontró el archivo para: {table}")

2026-02-18 16:05:18,249 - INFO - Loaded .env from c:\Users\USUARIO\Documents\Forecaster\Mi_Bunuelito\.env


✅ Cargada tabla: ventas_diarias (2963 registros)
✅ Cargada tabla: redes_sociales (2963 registros)


In [3]:
# Celda 3: Ejecución de Auditoría de Negocio
validator = BusinessValidator()
financial_report = validator.validate_all(data)

print(f"✅ Auditoría completada el {financial_report['timestamp']}")

2026-02-18 16:05:22,437 - INFO - Auditing business rules for ventas_diarias...
2026-02-18 16:05:22,444 - INFO - Auditing business rules for redes_sociales...


✅ Auditoría completada el 2026-02-18 16:05:22


## Hallazgos de Consistencia

In [4]:
# Celda 4: Resultados - Ventas Diarias
v_audit = financial_report["tables"].get("ventas_diarias", {})
if v_audit:
    print("📊 RESULTADOS AUDITORÍA FINANCIERA (VENTAS DIARIAS):")
    for rule, res in v_audit.items():
        status = "✅ CUMPLE" if res["success"] else "❌ FALLA"
        print(f"{status} - {rule}")
        if not res["success"]:
            print(f"   -> Errores detectados: {res.get('errors_count', res.get('violations_count', 0))}")

📊 RESULTADOS AUDITORÍA FINANCIERA (VENTAS DIARIAS):
✅ CUMPLE - total_units_consistency
✅ CUMPLE - promo_balance_consistency
✅ CUMPLE - price_viability
✅ CUMPLE - income_logic
❌ FALLA - cost_logic
   -> Errores detectados: 568
✅ CUMPLE - positive_gross_margin
✅ CUMPLE - utility_logic


In [5]:
# Celda 5: Resultados - Redes Sociales
r_audit = financial_report["tables"].get("redes_sociales", {})
if r_audit:
    print("📊 RESULTADOS AUDITORÍA (REDES SOCIALES):")
    for rule, res in r_audit.items():
        status = "✅ CUMPLE" if res["success"] else "❌ FALLA"
        print(f"{status} - {rule}")

📊 RESULTADOS AUDITORÍA (REDES SOCIALES):
✅ CUMPLE - investment_consistency


In [6]:
# Celda 6: Persistencia del Reporte (Fase 02)
output_dir = os.path.join("..", validator.reports_path)
os.makedirs(output_dir, exist_ok=True)
report_file = os.path.join(output_dir, "phase_02_financial_audit.json")

with open(report_file, "w", encoding="utf-8") as f:
    json.dump(financial_report, f, indent=4, ensure_ascii=False)

print(f"✅ Reporte financiero guardado en: {report_file}")

✅ Reporte financiero guardado en: ..\experiments/phase_02_financial_audit/artifacts\phase_02_financial_audit.json
